In [1]:
import mysql.connector as sql


In [2]:
connection = sql.connect(
    host="localhost",
    user="root",
    password="Ayoub1111",
    allow_local_infile=True,
    use_pure=True
)

In [3]:
cursor = connection.cursor()

In [4]:
cursor.execute("CREATE DATABASE IF NOT EXISTS product_sales")

In [5]:
cursor.execute('use product_sales')

In [6]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS discount (
        Month VARCHAR(255) NOT NULL,
        Discount_Band VARCHAR(255) NOT NULL,
        Discount INT NOT NULL
);
"""

cursor.execute(create_table_query)

In [7]:
connection.commit()

In [8]:
file_path = '/home/hp/Sql-Sales-Analysis/data/discount_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE discount
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Month, Discount_Band, Discount)
    """)


In [9]:
connection.commit()

In [10]:
cursor.execute("SHOW TABLES LIKE 'discount'")

In [11]:
result = cursor.fetchone()

In [12]:
if result:
    print("table exists")
else:
    print("table not exists")

table exists


In [13]:
table_name = 'discount'

In [14]:
query = f"SELECT * FROM {table_name}"

In [15]:
cursor.execute(query)
 

In [16]:
result = cursor.fetchall()

In [17]:
product_query = """
    CREATE TABLE IF NOT EXISTS Products (
        ProductID VARCHAR(50) NOT NULL,
        Product VARCHAR(255) NOT NULL,
        Category VARCHAR(255) NOT NULL,
        Cost_Price VARCHAR(255) NOT NULL,
        Sale_Price VARCHAR(255) NOT NULL,
        Brand VARCHAR(255) NOT NULL,
        Description VARCHAR(255) NOT NULL,
        Image_url VARCHAR(255) NOT NULL
);
"""

In [18]:
cursor.execute(product_query)

In [19]:
connection.commit()

In [21]:
file_path = '/home/hp/Sql-Sales-Analysis/data/Product_data.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE Products
    CHARACTER SET utf8mb4
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES 
    (ProductID, Product, Category, Cost_Price, Sale_Price, Brand, Description, Image_url)
    """)


In [22]:
connection.commit()

In [27]:
# create another tabe 
product_sales_table = """
    CREATE TABLE IF NOT EXISTS ProductSales (
        Date DATE,
        Customer_type VARCHAR(255) NOT NULL,
        Country VARCHAR(255) NOT NULL,
        Product VARCHAR(255) NOT NULL,
        Discount_Band VARCHAR(255) NOT NULL,
        Units_Sold INT NOT NULL
    );
"""
cursor.execute(product_sales_table)

In [28]:
file_path = '/home/hp/Sql-Sales-Analysis/data/product_sales.csv'
cursor.execute(
    f"""
    LOAD DATA LOCAL INFILE '{file_path}'
    INTO TABLE ProductSales
    CHARACTER SET utf8mb4
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES 
    (Date, Customer_type, Country, Product, Discount_Band, Units_Sold)
    """)

In [ ]:
connection.commit